In [1]:
import glob
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import csv

pd.options.display.float_format = '{:.5f}'.format
pd.set_option('display.max_columns', None)

In [2]:
use_counties = ['台北市', '高雄市', '新北市', '桃園市', '台中市', '台南市', '苗栗縣', '新竹縣', '基隆市',
       '屏東縣', '新竹市', '宜蘭縣', '花蓮縣', '嘉義市', '金門縣', '嘉義縣', '彰化縣', '雲林縣']
dfs = []
for pathto_file in glob.glob('../外部資料集/實價登錄/*/*_b.csv'):
    df = pd.read_csv(pathto_file, on_bad_lines='skip', skiprows=[1], quoting=csv.QUOTE_NONE)
    df['單價元平方公尺'].fillna(0)
    df['土地位置建物門牌'] = df['土地位置建物門牌'].apply(lambda x: (x.replace('臺北市', '台北市').replace('臺中市', '台中市').replace('臺南市', '台南市')))
    county = df['土地位置建物門牌'].apply(lambda x: (x+'123')[:3]).value_counts().index[0]
    if county not in use_counties:
        continue
    df['縣市'] = county
    avg_price = df.query('單價元平方公尺 > 1000')['單價元平方公尺'].median()
    avg_price = avg_price*0.1
    df['source'] = pathto_file.split('/')[-2]
    try:
        df = df.query(f'單價元平方公尺 > {avg_price}')
        dfs.append(df)
    except:
        pass
df = pd.concat(dfs).reset_index(drop=True)

In [3]:
use_sources = ['2021Q1', '2021Q2', '2021Q3', '2021Q4', '2022Q1', '2022Q2', '2022Q3', '2022Q4', '2023Q1', '2023Q2', '2023Q3']

print(df.shape)
df = df[df['source'].isin(use_sources)].copy().reset_index(drop=True)
print(df.shape)

(260348, 33)
(240097, 33)


In [4]:
set(use_counties) - set(df['縣市'].unique().tolist())

set()

In [5]:
df['縣市'].value_counts()

縣市
新北市    47337
桃園市    46609
台中市    45965
高雄市    23214
台南市    22095
新竹縣    11505
台北市    10368
苗栗縣     5682
彰化縣     5201
宜蘭縣     5055
雲林縣     3608
新竹市     3170
嘉義縣     2864
基隆市     2510
屏東縣     2001
嘉義市     1449
花蓮縣     1386
金門縣       78
Name: count, dtype: int64

In [6]:
df.groupby('鄉鎮市區')['單價元平方公尺'].mean()

鄉鎮市區
七堵區    79246.04348
七股區    51411.09091
三峽區    99773.46394
三星鄉    58945.92989
三民區   100297.24087
          ...     
麻豆區    70063.68760
鼓山區   110233.45405
龍井區    82707.38250
龍潭區    71048.39314
龜山區    96211.70483
Name: 單價元平方公尺, Length: 244, dtype: float64

In [7]:
pd.set_option('display.max_columns', None)
df

,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,移轉層次,總樓層數,建物型態,主要用途,主要建材,建築完成年月,建物移轉總面積平方公尺,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,建物現況格局-隔間,有無管理組織,總價元,單價元平方公尺,車位類別,車位移轉總面積平方公尺,車位總價元,備註,編號,建案名稱,棟及號,縣市,source,解約情形
0,萬華區,房地(土地+建物),台北市萬華區龍山段一小段0698-0000地號等1筆A1棟8樓,6.15000,商,NaN,NaN,1070705,土地1建物1車位0,八層,15,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土造,NaN,57.82000,1,0,1,有,無,9500000,164303.00000,NaN,0.00000,0,NaN,RPRSMLMKJIPFFBA08CB,景上川,NaN,台北市,2021Q1,NaN
1,萬華區,房地(土地+建物),台北市萬華區龍山段一小段0698-0000地號等1筆A1棟12樓,6.15000,商,NaN,NaN,1070815,土地1建物1車位0,十二層,15,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土造,NaN,57.82000,1,0,1,有,無,9500000,164303.00000,NaN,0.00000,0,售價包含裝潢或家電。；,RPTSMLMKJIPFFBA28CB,景上川,NaN,台北市,2021Q1,NaN
2,萬華區,房地(土地+建物),台北市萬華區龍山段一小段0698-0000地號等1筆A1棟13樓,6.15000,商,NaN,NaN,1070831,土地1建物1車位0,十三層,15,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土造,NaN,57.82000,1,0,1,有,無,10200000,176410.00000,NaN,0.00000,0,NaN,RPUSMLMKJIPFFBA38CB,景上川,NaN,台北市,2021Q1,NaN
3,萬華區,房地(土地+建物),台北市萬華區龍山段一小段0698-0000地號等1筆A1棟14樓,6.15000,商,NaN,NaN,1071102,土地1建物1車位0,十四層,15,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土造,NaN,57.82000,1,0,1,有,無,9700000,167762.00000,NaN,0.00000,0,售價包含裝潢或家電。；,RPVSMLMKJIPFFBA48CB,景上川,NaN,台北市,2021Q1,NaN
4,萬華區,房地(土地+建物),台北市萬華區龍山段一小段0698-0000地號等1筆A1棟15樓,6.15000,商,NaN,NaN,1071009,土地1建物1車位0,十五層,15,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土造,NaN,57.82000,1,0,1,有,無,9860000,170529.00000,NaN,0.00000,0,售價包含裝潢或家電。；,RPWSMLMKJIPFFBA58CB,景上川,NaN,台北市,2021Q1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240092,金寧鄉,房地(土地+建物),金門縣金寧鄉盤山村測段190、190-2、188-6地號,38.82000,其他,NaN,NaN,1120604,土地3建物1車位0,二層,5,華廈(10層含以下有電梯),住家用,鋼筋混凝土造,NaN,95.39000,3,2,2,有,無,6680000,70028.00000,NaN,0.00000,0,NaN,RPPNMLQJOHIGFAW96CB,金門縣金寧鄉盤山村測段190、190-2、188-6地號集合住宅新建工程,C22F棟15號,金門縣,2023Q3,NaN
240093,金寧鄉,房地(土地+建物),金門縣金寧鄉安西劃段,44.43000,其他,NaN,NaN,1120609,土地1建物1車位0,一層,5,華廈(10層含以下有電梯),住家用,鋼筋混凝土造,NaN,101.66000,3,2,2,有,無,8380000,82432.00000,NaN,0.00000,0,NaN,RPPNMLLLOHIGFAW66CB,光前首澤,C1棟0號,金門縣,2023Q3,NaN
240094,金寧鄉,房地(土地+建物),金門縣金寧鄉安西劃段,31.02000,其他,NaN,NaN,1120629,土地1建物1車位0,五層,5,華廈(10層含以下有電梯),住家用,鋼筋混凝土造,NaN,70.93000,3,2,2,有,無,4600000,64853.00000,NaN,0.00000,0,NaN,RPPNMLNJPHIGFAW76CB,光前首澤,C2棟0號,金門縣,2023Q3,NaN
240095,金湖鎮,房地(土地+建物),金門縣金湖鎮羅劃測段200，201地號,106.38000,住,NaN,NaN,1120728,土地2建物1車位0,全,3,透天厝,住家用,鋼筋混凝土造,NaN,208.86000,4,1,5,有,無,11180000,53529.00000,NaN,0.00000,0,NaN,RPPNMLNLQHIGFAW07CB,陽光寬庭,C棟6號,金門縣,2023Q3,NaN


In [8]:
df = df.query('單價元平方公尺 >= 1000')
df = df.query('單價元平方公尺 <= 2000000').reset_index(drop=True)

col = '主建物面積'
df[col] = 0

col = '陽台面積'
df[col] = 0

col = '附屬建物面積'
df[col] = 0

col = '車位面積'
df[col] = df['車位移轉總面積平方公尺'].values
scaler = StandardScaler()
scaler.fit(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
df[col] = scaler.transform(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
print(df[col].describe())

col = '建物面積'
df[col] = df['建物移轉總面積平方公尺'].values
scaler = StandardScaler()
scaler.fit(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
df[col] = scaler.transform(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
print(df[col].describe())

col = '土地面積'
df[col] = df['土地移轉總面積平方公尺'].values
scaler = StandardScaler()
scaler.fit(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
df[col] = scaler.transform(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
print(df[col].describe())

# col = '單價'
# df[col] = df['單價元平方公尺'].values
# scaler = StandardScaler()
# scaler.fit(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
# df[col] = scaler.transform(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1)) + 2
# print(df[col].describe())

count   240097.00000
mean         0.00000
std          1.00000
min         -1.72438
25%         -0.25186
50%          0.34977
75%          0.59185
max         37.72489
Name: 車位面積, dtype: float64
count   240097.00000
mean         0.00000
std          1.00000
min         -5.20245
25%         -0.56852
50%         -0.05443
75%          0.47478
max         71.95197
Name: 建物面積, dtype: float64
count   240097.00000
mean         0.00000
std          1.00000
min         -2.29632
25%         -0.58499
50%         -0.17606
75%          0.24691
max         36.81914
Name: 土地面積, dtype: float64


In [9]:
def get_year_from_str(x):
    try:
        x = int(x)
        if x < 1000101:
            return int(str(x)[0:2])
        else:
            return int(str(x)[0:3])
    except:
        return -1

def get_month_from_str(x):
    try:
        x = int(x)
        if x < 1000101:
            return int(str(x)[2:4])
        else:
            return int(str(x)[3:5])
    except:
        return -1

In [10]:
df['Trade_Year'] = df['交易年月日'].apply(get_year_from_str).astype(int) + 1911
df['Trade_Month'] = df['交易年月日'].apply(get_month_from_str).astype(int)
df['Trade_YearMonth'] = df['Trade_Year']*100 + df['Trade_Month']

In [11]:
pd.set_option('display.max_columns', None)
df = df.query('Trade_Year >= 2021 and Trade_Year <= 2022')

In [12]:
df['屋齡'] = 0

In [13]:
df['交易標的'].value_counts()

交易標的
房地(土地+建物)+車位    113711
房地(土地+建物)        32236
Name: count, dtype: int64

In [14]:
df['建物型態'].value_counts()

建物型態
住宅大樓(11層含以上有電梯)    100527
華廈(10層含以下有電梯)       25334
透天厝                 15654
辦公商業大樓               2090
店面(店鋪)               1234
廠辦                    388
其他                    351
套房(1房1廳1衛)            299
公寓(5樓含以下無電梯)           70
Name: count, dtype: int64

In [15]:
df = df[~df['總樓層數'].isna()].copy()
df = df[~df['總樓層數'].isna()].copy()
df = df.query('交易標的 != "土地"').copy()
df = df.reset_index(drop=True)
df['交易標的'].value_counts()

交易標的
房地(土地+建物)+車位    113711
房地(土地+建物)        32236
Name: count, dtype: int64

In [16]:
df['單價元平方公尺'].apply(lambda x: x / 60000) .describe()

count   145947.00000
mean         1.85692
std          0.97122
min          0.52173
25%          1.22973
50%          1.60682
75%          2.20677
max         14.36883
Name: 單價元平方公尺, dtype: float64

In [17]:
floor_dict = {
    '二十': 20,
    '三十': 30,
    '四十': 40,
    '五十': 50,
    '六十': 60,
    '七十': 70,
    '八十': 80,
    '九十': 90,
}
floor_dict2 = {
    '一': 1,
    '二': 2,
    '三': 3,
    '四': 4,
    '五': 5,
    '六': 6,
    '七': 7,
    '八': 8,
    '九': 9,
}
def get_floor(x):
    x = str(x)
    floor = 0
    for k, v in floor_dict.items():
        if x.startswith(k):
            x = x.replace(k, '')
            floor += v
    if x.startswith('十'):
        x = x.replace('十', '')
        floor += 10
    for k, v in floor_dict2.items():
        if x.startswith(k):
            x = x.replace(k, '')
            floor += v
    return floor

df['total_floor'] = df['總樓層數'].values
df['floor'] = df['移轉層次'].apply(get_floor)

In [18]:
df['total_floor'].describe()

count   145947.00000
mean        14.81390
std          7.93390
min          0.00000
25%          9.00000
50%         14.00000
75%         20.00000
max         42.00000
Name: total_floor, dtype: float64

In [19]:
def get_road_name(row):
    road_name = row['土地位置建物門牌']
    try:
        if road_name.startswith(row['縣市']):
            road_name = road_name.replace(row['縣市'], '')
        if road_name.startswith(row['鄉鎮市區']):
            road_name = road_name.replace(row['鄉鎮市區'], '')
    except:
        pass
    if '段' in road_name:
        road_name = road_name.split('段')[0] + '段'
    elif '路' in road_name:
        road_name = road_name.split('路')[0] + '路'
    elif '街' in road_name:
        road_name = road_name.split('街')[0] + '街'
    else:
        road_name = ""
    return road_name
    
df['路名'] = df.apply(lambda row: get_road_name(row), axis=1)

In [20]:
df = df.query('floor > 0').reset_index(drop=True)
df = df.query('total_floor > 0').reset_index(drop=True)
df = df.query('total_floor >= floor').reset_index(drop=True)
df = df.query('路名 != ""').reset_index(drop=True)

In [21]:
df.sample(5)

,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,移轉層次,總樓層數,建物型態,主要用途,主要建材,建築完成年月,建物移轉總面積平方公尺,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,建物現況格局-隔間,有無管理組織,總價元,單價元平方公尺,車位類別,車位移轉總面積平方公尺,車位總價元,備註,編號,建案名稱,棟及號,縣市,source,解約情形,主建物面積,陽台面積,附屬建物面積,車位面積,建物面積,土地面積,Trade_Year,Trade_Month,Trade_YearMonth,屋齡,total_floor,floor,路名
28962,中壢區,房地(土地+建物)+車位,桃園市中壢區福州二街及福星一街交叉口,30.06000,住,NaN,NaN,1100829,土地1建物1車位1,十層,12,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土造,NaN,158.19000,3,2,2,有,無,13330000,93253.00000,坡道平面,32.94000,1650000,NaN,RPSNMLQKRHGGFBH68CB,醴寓,A1-10號,桃園市,2021Q4,NaN,0,0,0,0.60175,0.42275,0.29208,2021,8,202108,0,12,10,福州二街
6906,竹北市,房地(土地+建物)+車位,新竹縣竹北市中興段421，422地號A棟6樓,23.27000,其他,NaN,NaN,1100308,土地2建物1車位1,六層,14,住宅大樓(11層含以上有電梯),住商用,混凝土造,NaN,175.29000,3,2,2,有,無,19670000,140749.00000,坡道平面,46.55000,1550000,NaN,RPXNMLLKOHGGFBJ58CB,馥邸,NaN,新竹縣,2021Q3,NaN,0,0,0,1.04086,0.71898,-0.01894,2021,3,202103,0,14,6,中興段
109388,三民區,房地(土地+建物)+車位,高雄市三民區自立一路和十全二路口,9.20000,商,NaN,NaN,1110915,土地2建物1車位1,十層,31,住宅大樓(11層含以上有電梯),住家用,鋼骨鋼筋混凝土造,NaN,156.15000,3,2,2,有,無,16270000,107105.00000,坡道平面,23.85000,2100000,NaN,RPPNMLLKIIHGFDE07CB,艾美國際城,B1棟10號,高雄市,2022Q4,NaN,0,0,0,0.25494,0.38636,-0.86436,2022,9,202209,0,31,10,自立一路
105723,太平區,房地(土地+建物)+車位,台中市太平區樹孝路67巷,17.59000,住,NaN,NaN,1110927,土地1建物1車位1,三層,15,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土造,NaN,116.55000,2,2,1,有,無,10480000,98879.00000,坡道平面,27.35000,1660000,NaN,RPPNMLOJIIHGFJB57CB,泓瑞綠雅圖,D棟3F-1號,台中市,2022Q4,NaN,0,0,0,0.39521,-0.37404,-0.31630,2022,9,202209,0,15,3,樹孝路
58312,龜山區,房地(土地+建物),桃園市龜山區文化一路A3-02F,4.83000,商,NaN,NaN,1101215,土地1建物1車位0,二層,26,住宅大樓(11層含以上有電梯),住商用,鋼筋混凝土造,NaN,60.50000,0,0,1,有,無,6080000,100496.00000,NaN,0.00000,0,NaN,RPQPMLNLKIGGFHH48CB,大亮泊,A棟03號,桃園市,2022Q1,NaN,0,0,0,-1.72438,-1.72368,-1.25876,2021,12,202112,0,26,2,文化一路


In [22]:
df['總樓層數'] = df['total_floor']
df['移轉層次'] = df['floor']

In [23]:
def get_car_cnt(x):
    x = x.split('車位')[-1]
    if x.isdigit():
        return int(x)
    return -999999

In [24]:
df['車位個數'] = df['交易筆棟數'].apply(get_car_cnt).astype(int)
df['車位個數'].describe()

count   127792.00000
mean         0.96825
std          1.11051
min          0.00000
25%          1.00000
50%          1.00000
75%          1.00000
max        300.00000
Name: 車位個數, dtype: float64

In [25]:
df['單價'] = df['單價元平方公尺'].apply(lambda x: x/58000)
df['單價'].describe()

count   127792.00000
mean         2.00838
std          1.03010
min          0.53972
25%          1.33844
50%          1.74573
75%          2.38397
max         14.86431
Name: 單價, dtype: float64

In [26]:
df = df.query('單價 > 0').copy().reset_index(drop=True)

In [27]:
df['格局_房'] = df['建物現況格局-房']
df['格局_廳'] = df['建物現況格局-廳']
df['格局_衛'] = df['建物現況格局-衛']
df['格局_隔間'] = df['建物現況格局-隔間'].map({'有': True, '無': False})
#df['格局_電梯'] = df['電梯'].map({'有': True, '無': False})
df['格局_管理'] = df['有無管理組織'].map({'有': True, '無': False})

In [28]:
print(df['主要用途'].value_counts())

主要用途
住家用        113210
見其他登記事項      5931
住商用          5491
商業用          2155
見使用執照         534
工業用           190
工商用           101
農業用            95
見其它登記事項        85
Name: count, dtype: int64


In [29]:
print(df.shape)
df = df.query('移轉層次 >= 2 and 移轉層次 <= 46')
print(df.shape)
df = df.query('總樓層數 >= 2 and 總樓層數 <= 68')
print(df.shape)
df = df.query('車位個數 >= 0 and 車位個數 <= 3')
print(df.shape)

(127792, 53)
(124716, 53)
(124716, 53)
(124547, 53)


In [30]:
use_cols = ['縣市', '鄉鎮市區', '路名', '建物型態', '主要建材', '總樓層數', '移轉層次', 
            '格局_房', '格局_廳', '格局_衛', '格局_隔間', '格局_管理',
            '主要用途', '車位個數', '屋齡', '土地面積', '建物面積', '車位面積', '主建物面積', '陽台面積', '附屬建物面積', '單價']

df[use_cols].to_csv('../外部資料集/實價登錄/external_gov_data_pre_sale.csv', index=False)
df[use_cols].head(10)

,縣市,鄉鎮市區,路名,建物型態,主要建材,總樓層數,移轉層次,格局_房,格局_廳,格局_衛,格局_隔間,格局_管理,主要用途,車位個數,屋齡,土地面積,建物面積,車位面積,主建物面積,陽台面積,附屬建物面積,單價
0,新北市,淡水區,淡金路二段,住宅大樓(11層含以上有電梯),鋼筋混凝土造,14,4,2,2,1,True,False,住家用,1,0,-0.41500,-0.31065,0.84182,0,0,0,1.23122
1,新北市,淡水區,淡金路二段,住宅大樓(11層含以上有電梯),鋼筋混凝土造,14,7,2,2,1,True,False,住家用,1,0,-0.41500,-0.31065,0.84182,0,0,0,1.26372
2,新北市,永和區,永和路一段,住宅大樓(11層含以上有電梯),鋼筋混凝土造,23,8,1,1,1,True,False,商業用,1,0,-1.00427,-0.82284,0.17793,0,0,0,3.22138
3,新北市,淡水區,淡金路二段,住宅大樓(11層含以上有電梯),鋼筋混凝土造,14,5,2,2,1,True,False,住家用,1,0,-0.41500,-0.31065,0.84182,0,0,0,1.23555
4,新北市,淡水區,淡金路二段,住宅大樓(11層含以上有電梯),鋼筋混凝土造,14,8,2,2,1,True,False,住家用,0,0,-0.41915,-1.21366,-1.72438,0,0,0,1.34393
5,新北市,淡水區,淡金路二段,住宅大樓(11層含以上有電梯),鋼筋混凝土造,14,10,2,2,1,True,False,住家用,0,0,-0.41915,-1.21366,-1.72438,0,0,0,1.37645
6,新北市,淡水區,淡金路二段,住宅大樓(11層含以上有電梯),鋼筋混凝土造,26,25,2,2,2,True,False,住家用,1,0,-0.32872,-0.13471,0.84182,0,0,0,1.48788
7,新北市,淡水區,淡金路二段,住宅大樓(11層含以上有電梯),鋼筋混凝土造,14,3,2,2,1,True,False,住家用,1,0,-0.42748,-0.33751,0.84182,0,0,0,1.36643
8,新北市,淡水區,淡金路二段,住宅大樓(11層含以上有電梯),鋼筋混凝土造,14,11,2,2,1,True,False,住家用,1,0,-0.41086,-0.30391,0.84182,0,0,0,1.35566
9,新北市,淡水區,淡金路二段,住宅大樓(11層含以上有電梯),鋼筋混凝土造,14,9,2,2,1,True,False,住家用,0,0,-0.41915,-1.21366,-1.72438,0,0,0,1.35478
